
# ***INSTALLATION OF SOME LIBRARIES***



In [ ]:
!pip install langchain_groq

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_huggingface

# **IMPORT THE LIBRARIES**

In [ ]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBasedLoader
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")


# ***FETCHING THE DATA BY WEBSCRAP***

In [ ]:
import bs4
data=WebBasedLoader(web_paths="www.huggingface.com",
                    bs_kwargs=dict(
                        parse_only=bs4.SoupStrainer(
                            class_=("post-content" , "post-title" , "post-header")
                        )
                    ))

# ***SPLIT THE DATA INTO CHUNK***

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500  ,chunk_overlap=100)
final_document=splitter.split_document(data)


# ***LOADING THE EMBEDDING***

In [ ]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="ganna-2b")



In [ ]:
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key , model_name="lama3-8b-8192")

# ***CREATE_THE VECTORSTORE***

In [ ]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents=final_document,embedding=embedding)
retriever=vectorstore.as_retriever()
retriever

In [ ]:
system_prompt=
("""
 you are an assistant for quetion -annwering tasks.
 use the follwing pices o contect to answer the qquetion of dont kno the answer
 say that you dont know use maximum three sentences maximun and keep the answer concise {context}
 """ )

# ***CREATE_A_PROMPT_TEMPLATE***

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

promt=ChatPromptTemplate(
    [
        ("system",system_prompt),
        ("human", "{input}"),
    ]
)

# ***CREATE THE RETRIVEAL CHAIN***

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_document_chain
quetion_answer_chain=create_stuff_document_chain(llm, promt)
rag_chain=create_retrieval_chain(retriever ,quetion_answer_chain)

In [ ]:
responce=rag_chain.invoke("input", "what is info in the given text")

In [ ]:
#adding the history in that

# ***GIVEN CODE IS FOR THE ADDING  THE CHAT-HISTORY***

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


contextualize_q_system_prompt=(
    "given a chat history and the leest user wuritom wihhc mrhg refrence in th chat hidtory formulate  a standalone quetion which can be understood without and chat history do not answer the qieto jusgt reformulate it id needed and othete return it as is"
)

In [ ]:
context_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system" ,contextualize_q_system_prompt ),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [ ]:
history_aware_retriver=create_history_aware_retriever(llm , retriever , context_q_prompt)

In [ ]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_document_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriver,question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
quetion="what is the self-reflection"
responce1=rag_chain.invoke({"input":quetion , "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=quetion),
        AIMessage(content=responce1["answer"])
    ]
)

qution_2="tell me more about it"
responce2=rag_chain.invoke({"input":quetion , "chat_history":chat_history})
print(responce2["answer"])

In [ ]:
chat_history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}

In [ ]:
def get_session_history(session_id : str)-->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]=ChatMessageHistory()
  return store[session_id]

In [ ]:
conversional_rag_chain=RunnableWithMessageHistory(
    rag_chain ,
    get_session_history,
    inout_message_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    )

In [ ]:
conversional_rag_chain.invoke(
    {"input":"what is the task decomposition"},
    config={
        "configurable":{"session_id":"abc123"}

    },
    )["answer"]

In [ ]:
conversional_rag_chain.invoke(
    {"input":"what are the comman way og doing it"},
    config={"configureble":{"session_id":"abc123"}}
)["answer"]